In [291]:
import pandas as pd
import numpy as np
import pickle

from classes.ClusterModel import ClusterModel

import json

In [292]:
modelo_clientes = pickle.load(open('../models/kmeans_clientes_FINAL.pkl', 'rb'))
modelo_fixa = pickle.load(open('../models/recommendation_fixa_FINAL.pkl', 'rb'))
modelo_fundos = pickle.load(open('../models/recommendation_fundos_FINAL.pkl', 'rb'))

In [293]:
clientes = pd.read_csv('../data/processados/clientes_final.csv')
fixa = pd.read_csv('../data/processados/produto_fixa_final.csv')
fundos = pd.read_csv('../data/processados/produto_fundos_final.csv')

In [294]:
full_fundos = pd.read_csv('../data/processados/produtos_fundos.csv')
full_fixa = pd.read_csv('../data/processados/produtos_fixa.csv')

In [295]:
entry = '{"Id":"$2b$10$.DCvvJ9WSDpqVP2xXjeSPeR6I3TrUCl/MAePvQQcvloiuSPbG0TpW","P_investidor":"conservador", "b_renda_mensal":"(-50.0, 10000.0]", "tempo_dinheiro_inves":"inder","resgate_dia":0, "inves_renda_mensal":10}'

In [296]:
jsn = json.loads(entry)

In [297]:
data = pd.DataFrame([jsn], columns=jsn.keys())

In [298]:
cluster = clientes[clientes['Id'] == data['Id'].values[0]]['cluster']

In [299]:
recomendacao_fixa = modelo_fixa.query(f'customerId == {cluster.values[0]}').iloc[0]['produtoId']
recomendacao_fundos = modelo_fundos.query(f'customerId == {cluster.values[0]}').iloc[0]['produtoId']

In [300]:
full_fundos = full_fundos[['ProdutoId', 'NomeInvestimento__c', 'FiltroValorMinimo__c']]

In [301]:
full_fixa = full_fixa[['ProdutoId', 'NomeInvestimento__c', 'NomeProduto__c']]

In [302]:
produtos_fixa = fixa.query(f'cluster == {recomendacao_fixa}').iloc[range(0,2)]

In [303]:
produtos_fundos = fundos.query(f'cluster == {recomendacao_fundos}').iloc[range(0,2)]

In [304]:
fundo =json.dumps({"fundo":produtos_fundos.merge(full_fundos, left_on='ProdutoId', right_on='ProdutoId', how='left').to_dict('records')})

In [305]:
fixa =json.dumps({"fixa":produtos_fixa.merge(full_fixa, left_on='ProdutoId', right_on='ProdutoId', how='left').to_dict('records')})

In [306]:
fundo +',' +fixa

'{"fundo": [{"FamiliaCarteira__c": "Renda Fixa", "RiscoAtivo__c": 2.0, "cluster": -1, "ProdutoId": "a0W3u00000AdEFNEA3", "NomeInvestimento__c": "CA INDOSUEZ DI MASTER FIRF REF. DI LP", "FiltroValorMinimo__c": 1000.0}, {"FamiliaCarteira__c": "Renda Fixa", "RiscoAtivo__c": 2.0, "cluster": -1, "ProdutoId": "a0W3u00000AdGFdEAN", "NomeInvestimento__c": "AZ QUEST YIELD FIC FIRF LP", "FiltroValorMinimo__c": 10000.0}]},{"fixa": [{"b_dias_vencimento": "(503.2, 1007.4]", "FiltroValorMinimo__c": 1000.0, "DescricaoCarencia__c": "Qualquer dia", "cluster": -1, "ProdutoId": "a0W3u000009PBsjEAG", "NomeInvestimento__c": "CDB", "NomeProduto__c": "CDB POS 1"}, {"b_dias_vencimento": "(503.2, 1007.4]", "FiltroValorMinimo__c": 1000.0, "DescricaoCarencia__c": "Qualquer dia", "cluster": -1, "ProdutoId": "a0W3u000009PC0WEAW", "NomeInvestimento__c": "CDB", "NomeProduto__c": "CDB POS"}]}'

In [307]:
json.loads(fundo)

{'fundo': [{'FamiliaCarteira__c': 'Renda Fixa',
   'RiscoAtivo__c': 2.0,
   'cluster': -1,
   'ProdutoId': 'a0W3u00000AdEFNEA3',
   'NomeInvestimento__c': 'CA INDOSUEZ DI MASTER FIRF REF. DI LP',
   'FiltroValorMinimo__c': 1000.0},
  {'FamiliaCarteira__c': 'Renda Fixa',
   'RiscoAtivo__c': 2.0,
   'cluster': -1,
   'ProdutoId': 'a0W3u00000AdGFdEAN',
   'NomeInvestimento__c': 'AZ QUEST YIELD FIC FIRF LP',
   'FiltroValorMinimo__c': 10000.0}]}

In [308]:
json.loads(fixa)

{'fixa': [{'b_dias_vencimento': '(503.2, 1007.4]',
   'FiltroValorMinimo__c': 1000.0,
   'DescricaoCarencia__c': 'Qualquer dia',
   'cluster': -1,
   'ProdutoId': 'a0W3u000009PBsjEAG',
   'NomeInvestimento__c': 'CDB',
   'NomeProduto__c': 'CDB POS 1'},
  {'b_dias_vencimento': '(503.2, 1007.4]',
   'FiltroValorMinimo__c': 1000.0,
   'DescricaoCarencia__c': 'Qualquer dia',
   'cluster': -1,
   'ProdutoId': 'a0W3u000009PC0WEAW',
   'NomeInvestimento__c': 'CDB',
   'NomeProduto__c': 'CDB POS'}]}

In [309]:
st = f'"data":[{fixa},{fundo}]'

In [310]:
result = '{' + st + '}'

In [311]:
json.loads(result)

{'data': [{'fixa': [{'b_dias_vencimento': '(503.2, 1007.4]',
     'FiltroValorMinimo__c': 1000.0,
     'DescricaoCarencia__c': 'Qualquer dia',
     'cluster': -1,
     'ProdutoId': 'a0W3u000009PBsjEAG',
     'NomeInvestimento__c': 'CDB',
     'NomeProduto__c': 'CDB POS 1'},
    {'b_dias_vencimento': '(503.2, 1007.4]',
     'FiltroValorMinimo__c': 1000.0,
     'DescricaoCarencia__c': 'Qualquer dia',
     'cluster': -1,
     'ProdutoId': 'a0W3u000009PC0WEAW',
     'NomeInvestimento__c': 'CDB',
     'NomeProduto__c': 'CDB POS'}]},
  {'fundo': [{'FamiliaCarteira__c': 'Renda Fixa',
     'RiscoAtivo__c': 2.0,
     'cluster': -1,
     'ProdutoId': 'a0W3u00000AdEFNEA3',
     'NomeInvestimento__c': 'CA INDOSUEZ DI MASTER FIRF REF. DI LP',
     'FiltroValorMinimo__c': 1000.0},
    {'FamiliaCarteira__c': 'Renda Fixa',
     'RiscoAtivo__c': 2.0,
     'cluster': -1,
     'ProdutoId': 'a0W3u00000AdGFdEAN',
     'NomeInvestimento__c': 'AZ QUEST YIELD FIC FIRF LP',
     'FiltroValorMinimo__c': 10000.0